# Extração das ementas da Alura
Este script fora desenvolvido para extrair um arquivo estruturado contendo as seguintes informações do site da Alura:
* Escolas(Progração, Front-End)
* Tipo de Curso (Java, PHP, C# e outros) 
* Curso(Algoritmos, por exemplo)
* Carga Horária do curso 
* Ementa, com aulas e seus assuntos

In [22]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import json, io
url='https://www.alura.com.br'
#response=urlopen(url)
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [24]:
def aulas(url, link):
    r = requests.get(url+link)    
    bs = BeautifulSoup(r.text, 'html.parser')
    link = bs.find_all(class_='courseInfo-detailed-content-item')
    aulas=[{
        'aula':linha.h3.text,
        'topicos': [ i.text for i in linha.find_all(class_='courseInfo-detailed-content-item-topic')]
    } for linha in link]
    return aulas

In [25]:
def pagina(url, link):
    r = requests.get(url+link)    
    bs = BeautifulSoup(r.text, 'html.parser')
    link = url+bs.find(class_='school-link-all-courses').get('href')
    return {'link' : link, 'cursos' : cursos(link)}

def cursos(link):
    cursos=[]
    r = requests.get(link)    
    bs = BeautifulSoup(r.text, 'html.parser')
    container=bs.find(class_='lista-subcategorias__subcategorias subcategoria__container')
    categorias=container.find_all(class_='subcategoria lista-subcategorias__subcategoria')
    for categoria in categorias:
        grupo_curso={
            'grupo_curso': categoria.h2.text,
            'link_grupo_curso': url+categoria.a.get('href'),
            'cursos': [
                {
                    'curso' : i.span.text,
                    'carga_horaria': i.find(class_='card-curso__carga').text.strip(),
                    'link_curso': url+i.get('href'),
                    'aulas' : aulas(url, i.get('href'))
                    
                }for i in categoria.find_all(class_='card-curso')]    
        }
        cursos.append(grupo_curso)
    return cursos

# Fazendo a extração dos links das escolas a partir da Home Page
Neste passo consigo obter o nome da escola e o link que adentra aos seus cursos e salvo em arquivo JSON.

In [26]:
print('Iniciando a extração das informações e será printado quando ficar pronto')
alura=soup.find(class_='container --items')
escolas=alura.find_all(class_='careers__list__item')
linha=[]
for escola in escolas:
    dicionario={'escola': escola.h3.text, 
                'link_escola': url+escola.find(class_='careers__list__item__see-more__link').get('href'),
                'link_cursos':pagina(url,escola.find(class_='careers__list__item__see-more__link').get('href')),
               }
    linha.append(dicionario)
with open('alura.json', 'w', encoding='utf-8') as arquivo:
    json.dump({'alura':linha}, arquivo)
print('Pronto e salvo o arquivo: alura.json')

Iniciando a extração das informações e será printado quando ficar pronto
Pronto e salvo o arquivo: alura.json
